# Tests

## Create a stream in redis using Aia Utilities

In [ ]:
from zoneinfo import ZoneInfo
import aia_utilities as au
from datetime import datetime
import pytz

ru = au.Redis_Utilities()
tm = au.Time_Management()

# stream = 'test_stream'
stream = 'prices'


## Populate it with n entries ordered or random timestamps

In [ ]:
from time import sleep
from datetime import datetime, timedelta
import pytz  
import random

ordered = False
iterations = 100

for i in range(iterations):

    if ordered:
        timestamp = tm.datetime_to_string(datetime.now())
    else:
        timestamp = tm.datetime_to_string(random.uniform(datetime.now() + timedelta(minutes=5), datetime.now() - timedelta(minutes=5)))
    item1 = {"timestamp": timestamp, "price": 1.0 + i, "currency": "USD"}
    ru.write(stream, item1)

## Show the stream

In [ ]:
sample = 100
ru.show(stream, sample)

## Read all entries and order them or not

In [ ]:
ordered = True  # False

return_dict = ru.read_all(stream, order=ordered)

is_ordered = all(return_dict[i]["timestamp"] <= return_dict[i + 1]["timestamp"] for i in range(len(return_dict) - 1))
print(f"{len(return_dict)} items, ordered: {is_ordered}")

# Delete the stream

In [ ]:
ru.delete(stream)

## Test read_each by inserting items randomly and reading them

In [ ]:
# While you read each, also insert some entries once every 10 seconds, in a separate thread
import threading
from time import sleep    
import random

ru.delete('test_stream')
def insert_entries():
    # make the range 0-5 random
    while True:
        sleep(random.randint(0, 5))
        # create a now timestamp with 6 decimals for new york time
        now = datetime.now(pytz.timezone("America/New_York"))
        timestamp = now.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        item1 = {"timestamp": timestamp, "price": random.randint(0, 100), "currency": "USD"}
        ru.write(stream, item1)

# While you read each, also insert some entries once every 10 seconds, in a separate thread
thread = threading.Thread(target=insert_entries)
thread.start()

for entry in ru.read_each(stream):
    print(entry)